This notebook demonstrates several geospatial processing tasks, beginning with the conversion of GDB and GML files into the FlatGeobuf (FGB) format for improved handling of spatial data. The processed data is then queried and filtered using DuckDB, which efficiently handles spatial queries on the local machine.

Two primary filtering approaches are applied to the locally stored dataset:

* Direct Filtering:
The first two filtering methods operate directly on the file already saved on the local machine. One method quickly retrieves an initial subset (built areas) to provide a quick overview, while the other selects all rows within a 500-meter radius of a chosen point, capturing entries for all three species.

* Streaming-Based Filtering:
The remaining two methods perform nearly identical filtering as the direct methods but include sleep delays to simulate streaming. This demonstrates how the data can be processed incrementally in batches as it is ingested over time.

The notebook presents the filtered results in both tabular format and interactive map form, offering clear visualizations of the spatial filtering outcomes.

# Import

In [1]:
import os
import geopandas as gpd
import fiona
import duckdb
import folium
from shapely import wkt
from shapely.geometry import Point, Polygon
import time
import random
import pandas as pd
from IPython.display import display, clear_output
from pathlib import Path

import ipywidgets as widgets


# Constant

In [2]:
ROOT_DIR = Path().resolve()

# Functions and classes


In [3]:
# Class that convert bytes to mb and kb
class FileSizeConverter:
    def human_readable_size(self, size):
        # If the size is greater than 100,000 bytes -> convert to megabytes
        if size > 100000:
            size_mb = size / (1024 ** 2)
            return f"{size_mb:.2f} MB"
        else:
            # Otherwise -> convert to kilobytes
            size_kb = size / 1024
            return f"{size_kb:.2f} KB"

In [4]:
# Function that connects to DuckDB
def get_duckdb_connection():
    con = duckdb.connect()
    # Install and load the spatial extension for DuckDB.
    con.execute("INSTALL spatial; LOAD spatial;")
    return con

In [ ]:
# Function that export the result to flatGeoBuf.
def export_gdf_with_stats(gdf, output_path, original_file_path):
    # Start the timer to measure export time
    start_time = time.time()

    # Export the GeoDataFrame to a FlatGeoBuf file.
    gdf.to_file(output_path, driver="FlatGeobuf")

    # Stop the timer after export.
    end_time = time.time()
    elapsed_time = end_time - start_time # Calculate the total time

    # Get the file size in bytes
    orig_size_bytes = os.path.getsize(original_file_path)
    exported_size_bytes = os.path.getsize(output_path)

    # Convert file sizes to human-readable format using fileSizeConverter
    converter = FileSizeConverter()
    orig_size_str = converter.human_readable_size(orig_size_bytes)
    exported_size_str = converter.human_readable_size(exported_size_bytes)

    # Print out information about the downloaded file and original file.
    print(f"GeoDataFrame er lagret til: {output_path}")
    print(f"Tid brukt på eksport: {elapsed_time:.2f} sekunder")
    print(f"Original fil ({original_file_path}) størrelse: {orig_size_str}")
    print(f"Størrelse på eksportert fil: {exported_size_str}")

In [ ]:
# Function that determine abcolor based on the tree species value
def get_color(treslag):
    try:
        value = int(float(treslag))
    except Exception:
        value = None
    color_map = {
        31: "blue",
        32: "green",
        33: "red",
        39: "purple"
    }
    return color_map.get(value, "gray")

# function for GeoJSON layer that uses the treslag property to set feature colors
def style_function(feature):
    treslag = feature['properties'].get('treslag')
    color = get_color(treslag)
    return {
        'fillColor': color,
        'color': color,
        'weight': 1,
        'fillOpacity': 0.5,
    }

# Converting to FGB file format

In [8]:
# Function that converting file til FGB
def convert_to_flatgeobuf(input_file, output_file, layer=None):
    # Get the file extension in lowercase
    file_ext = os.path.splitext(input_file)[1].lower()

    # For GML files, the layer parameter must be specified.
    if file_ext == '.gml':
        if layer is None:
            raise ValueError("For GML-files, 'layer' must be specified (e.g., 'ArealressursFlate').")
        gdf = gpd.read_file(input_file, layer=layer)
    # For GDB-files, the layer can be automatically determined if not specified.
    elif file_ext == '.gdb':
        if layer is None:
            # List all available layers and choose the first one.
            layers = fiona.listlayers(input_file)
            print("Tilgjengelige lag i GDB:", layers)
            layer = layers[0]
        gdf = gpd.read_file(input_file, layer=layer)
    else:
        raise ValueError(f"Ukjent eller ikke støttet filtype: {file_ext}")

    # Filter the geodataframe to retain only polygons (polygon and multipolygon)
    gdf = gdf[gdf.geom_type.isin(['Polygon', 'MultiPolygon'])]
    print("Geometrityper etter filtrering:")
    print(gdf.geom_type.value_counts())

    # Export the filtered GeoDataFrame to a FlatGeobuf (FGB) file.
    gdf.to_file(output_file, driver="FlatGeobuf")
    print(f"Lagret polygoner som {output_file}")

    return gdf
# Example usage:
input_gml = ROOT_DIR  / "data" / "42_25832_ar50_gml.gml"
input_gdb = ROOT_DIR / "data" / "AR50_gdb.gdb"
output_fgb = ROOT_DIR / "data" / "AR50_flater.fgb"
# For the GDB file, you must specify ar50 as the layer
# For the GML file, you must specify ArealressursFlate as the layer.
convert_to_flatgeobuf(input_gml, output_fgb, layer="ArealressursFlate")

Geometrityper etter filtrering:
Polygon    179790
Name: count, dtype: int64
Lagret polygoner som /Users/ineantonsen/Desktop/Bachelor-V25/repo/kaidata_geolake/AR50_brukerhistorier/data/AR50_flater.fgb


,gml_id,lokalId,navnerom,informasjon,områdeId,originalDatavert,kopidato,målemetode,oppdateringsdato,arealtype,skogbonitet,treslag,jordbruk,vegetasjonsdekke,kartstandard,geometry
0,idfbb8c036-19e3-4a9a-b2f6-0c2d35695c1d,176214,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,13,31,98,98,AR50,"POLYGON ((485218.613 6517125.999, 485201.686 6..."
1,id36b5f040-baf2-4727-b0a1-cc90cb4bd608,176213,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,11,99,98,98,AR50,"POLYGON ((485586.131 6519504.745, 485607.835 6..."
2,ide33f3b18-38c2-4514-ae13-ca2df0a2e41a,176212,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,60,12,31,98,98,AR50,"POLYGON ((487006.634 6519259.077, 486992.389 6..."
3,idd556feed-1ea8-48c4-af32-9ad8e3da9fe7,176211,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,60,11,39,98,98,AR50,"POLYGON ((483970.068 6519640.093, 483990.43 65..."
4,id5f39e277-26d6-42b0-bacf-f81ac3a628d2,176210,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,11,31,98,98,AR50,"POLYGON ((485413.573 6520531.763, 485458.321 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179785,id8d863682-6cb3-4c6b-9190-8eefa6cf6633,5,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,60,11,39,98,98,AR50,"POLYGON ((358617.147 6442580.922, 358631.381 6..."
179786,idff57878f-0fbb-4ee5-9fa5-d8e364e9ff58,4,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,13,31,98,98,AR50,"POLYGON ((358002.134 6441284.427, 357998.139 6..."
179787,idbbd694b0-068f-4308-ab6e-8c5d3f34e011,3,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,50,98,39,98,55,AR50,"POLYGON ((357579.6 6444891.118, 357578.268 644..."
179788,idce0ff6b8-8fa6-4cb7-8476-08e2415bb4ad,2,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,20,98,98,24,98,AR50,"POLYGON ((357225.405 6444709.267, 357232.189 6..."


# Filtering

## Top 10 developed areas in Kristiansand (userstory 2)

The filtering retrieves data from a dataset by selecting rows based on user-provided input for "områdeId" (for example, 4204 for Kristiansand) and "arealtype" (for example, 10 for built-up areas). It then limits the output to the first 10 rows, providing a focused snapshot of the data for that specific region and type

In [14]:
arealtype_widget = widgets.IntText(value=10, description="Areal type:")
områdeid_widget  = widgets.IntText(value=4204, description="Area ID:")
knapp = widgets.Button(description="Load data")
output = widgets.Output()

def on_click(b):
    with output:
        output.clear_output()
        # Establish DuckDB connection
        con = get_duckdb_connection()

        # Read user inputs
        ar = arealtype_widget.value
        om = områdeid_widget.value

        # SQL query using the user-provided filters
        query = f"""
            SELECT
                "gml_id",
                "områdeId",
                "arealtype",
                ST_AsText("geom") AS geometry_wkt
            FROM ST_Read('data/AR50_flater.fgb')
            WHERE "arealtype" = {ar} AND "områdeID" = {om}
            LIMIT 10
        """
        df = con.execute(query).fetchdf()

        # Convert WKT strings to geometry and build a GeoDataFrame
        df['geometry'] = df['geometry_wkt'].apply(wkt.loads)
        gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:25832")

        # Export the filtered GeoDataFrame to FlatGeoBuf
        export_gdf_with_stats(gdf, "data/filter1_direct.fgb", "data/AR50_flater.fgb")

        # Compute the centroid in a metric projection, then convert to WGS84
        gdf_proj = gdf.to_crs("EPSG:3857")
        centroid_proj = gdf_proj.geometry.union_all().centroid
        centroid_wgs = gpd.GeoSeries([centroid_proj], crs="EPSG:3857") \
                         .to_crs("EPSG:4326").iloc[0]
        center_lat, center_lon = centroid_wgs.y, centroid_wgs.x

        # Reproject the GeoDataFrame to WGS84 for display
        gdf_wgs = gdf.to_crs("EPSG:4326")

        # Display the raw DataFrame
        display(df)

        # Create and display the Folium map with popups
        m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
        popup = folium.GeoJsonPopup(
            fields=["arealtype", "områdeId"],
            aliases=["Areal type:", "Area ID:"],
            localize=True,
            labels=True,
            style="background-color: white;"
        )
        folium.GeoJson(
            gdf_wgs,
            style_function=lambda feature: {
                'fillColor': 'blue',
                'color': 'blue',
                'weight': 1,
                'fillOpacity': 0.5
            },
            popup=popup
        ).add_to(m)
        display(m)

# Wire up the button and display the widgets
knapp.on_click(on_click)
display(widgets.VBox([
    widgets.HBox([arealtype_widget, områdeid_widget]),
    knapp,
    output
]))

## Tree species identification by coordinates: A radius search (userstory 1)

This filter lets the user specify a center point by entering its latitude and longitude. It then generates a 500-meter buffer around that user-defined point and examines all features within that area to report which tree species (treslag) are present.

### Cleaning the data

In this section, the data cleaning process addresses non-informative values in the dataset. Specifically, numerical codes such as 98 and 99, which indicate "not registered", are converted to null values. This step is crucial because it ensures that these placeholder values are ignored during subsequent queries. Without this conversion, queries such as aggregating or filtering based on tree species in a given area could yield incorrect or misleading results. By setting these values to null, the filtering operations can accurately reflect the real data, providing reliable outcomes for analyses.

In [19]:
# Connection to DuckDB
con = get_duckdb_connection()

con.execute("""
CREATE OR REPLACE TEMPORARY VIEW renset_data AS
SELECT
  CASE WHEN arealtype IN (98, 99) THEN NULL ELSE arealtype END AS arealtype,
  CASE WHEN skogbonitet IN (98, 99) THEN NULL ELSE skogbonitet END AS skogbonitet,
  CASE WHEN treslag IN (98, 99) THEN NULL ELSE treslag END AS treslag,
  CASE WHEN jordbruk IN (98, 99) THEN NULL ELSE jordbruk END AS jordbruk,
  CASE WHEN vegetasjonsdekke IN (98, 99) THEN NULL ELSE vegetasjonsdekke END AS vegetasjonsdekke,
  *
FROM ST_READ('data/AR50_flater.fgb')
""")

### Query and visualization

In [39]:
# — Widgets for filters, center og buffer —
områdeid_widget    = widgets.IntText(
    value=4204,
    description="Område-ID:",
    style={'description_width':'initial'}
)
treslag_widget     = widgets.IntText(
    value=0,
    description="Treslag (0=alle):",
    style={'description_width':'initial'}
)
center_lat_widget  = widgets.FloatText(
    value=58.1743128719827,
    description="Breddegrad:",
    style={'description_width':'initial'}
)
center_lon_widget  = widgets.FloatText(
    value=8.01122450699552,
    description="Lengdegrad:",
    style={'description_width':'initial'}
)
buffer_widget      = widgets.IntSlider(
    value=500,
    min=0,
    max=2000,
    step=50,
    description="Buffer (m):",
    style={'description_width':'initial'}
)
load_button        = widgets.Button(description="Last inn og buffér")
output             = widgets.Output()

def on_click(b):
    with output:
        output.clear_output()

        om         = områdeid_widget.value
        treslag_v  = treslag_widget.value
        lat        = center_lat_widget.value
        lon        = center_lon_widget.value
        buffer_m   = buffer_widget.value  # bruker-valgt buffer

        # Dynamisk treslag-clause
        treslag_clause = ""
        if treslag_v != 0:
            treslag_clause = f"  AND treslag = {treslag_v}\n"

        # Bygg buffer rundt brukerens punkt
        pt = Point(lon, lat)
        buf_wkt = (
            gpd.GeoSeries([pt], crs="EPSG:4326")
               .to_crs("EPSG:25832")
               .iloc[0]
               .buffer(buffer_m)  # bruker-valgt radius
               .wkt
        )
        print(f"Bruker buffer = {buffer_m} m rundt ({lat}, {lon})")

        # SQL-spørring mot viewet
        query = f"""
        SELECT
          gml_Id,
          lokalId,
          treslag,
          områdeId,
          ST_AsText(
            ST_Intersection(geom, ST_GeomFromText('{buf_wkt}', false))
          ) AS geometry_wkt
        FROM renset_data
        WHERE treslag IS NOT NULL
          AND ST_Intersects(geom, ST_GeomFromText('{buf_wkt}', false))
        {treslag_clause};
        """

        df = con.execute(query).fetchdf()

        # Til GeoDataFrame & eksport
        df['geometry'] = df['geometry_wkt'].apply(wkt.loads)
        gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:25832")
        export_gdf_with_stats(gdf, "data/filter2_direct.fgb", "data/AR50_flater.fgb")

        # Vis tabell
        display(df)

        # Render kartet med style_function
        gdf_wgs = gdf.to_crs("EPSG:4326")
        m = folium.Map(location=[lat, lon], zoom_start=15)
        popup = folium.GeoJsonPopup(
            fields=['treslag'],
            aliases=['Treslag:'],
            localize=True,
            labels=True,
            style="background-color:white;"
        )
        folium.GeoJson(
            gdf_wgs.to_json(),
            style_function=style_function,
            popup=popup
        ).add_to(m)
        display(m)

load_button.on_click(on_click)

# Vis alle widgets, inkludert buffer-slider
display(widgets.VBox([
    widgets.HBox([områdeid_widget, treslag_widget]),
    widgets.HBox([center_lat_widget, center_lon_widget]),
    buffer_widget,
    load_button,
    output
]))

# Testing using sleep to mimic network transfer delays

### Sleep function

In [31]:
# Simulates streaming of geospatial data by splitting the full dataset into smaller batches with an artificial delay
def stream_geo_data(con, query, batch_size=5):
    df_all = con.execute(query).fetchdf()
    for idx in range(0, len(df_all), batch_size):
        batch = df_all.iloc[idx:idx+batch_size].copy()
        delay = random.uniform(0.5, 1.5)
        time.sleep(delay)
        print(f"Hentet batch {idx//batch_size+1} med {len(batch)} rader. Neste om {delay:.2f} sek…")
        yield batch

## Filtering: Top 10 developed areas in Kristiansand

### Query and visualization

In [32]:
# — Widgets —
arealtype_widget  = widgets.IntText(value=10, description="Arealtype:", style={'description_width':'initial'})
områdeid_widget   = widgets.IntText(value=4204, description="Område-ID:", style={'description_width':'initial'})
batch_slider      = widgets.IntSlider(value=2, min=1, max=10, description="Batch‐størrelse:")
load_button       = widgets.Button(description="Start streaming")
output            = widgets.Output()

def on_click(btn):
    with output:
        output.clear_output()
        ar = arealtype_widget.value
        om = områdeid_widget.value
        bs = batch_slider.value

        # Build and run streaming query
        query = f"""
            SELECT
              gml_id,
              områdeId,
              arealtype,
              ST_AsText(geom) AS geometry_wkt
            FROM renset_data
            WHERE arealtype = {ar}
              AND områdeId  = {om}
            LIMIT 10;
        """
        print(f"Starter streaming med arealtype={ar}, områdeId={om}\n")
        batches = []
        for batch in stream_geo_data(con, query, batch_size=bs):
            batch['geometry'] = batch['geometry_wkt'].apply(wkt.loads)
            batches.append(batch)

        # Combine and show
        final_df = pd.concat(batches, ignore_index=True)
        print("\nAlle rader hentet:")
        display(final_df)

        # To GeoDataFrame, export, centroid & map
        gdf = gpd.GeoDataFrame(final_df, geometry='geometry', crs="EPSG:25832")
        export_gdf_with_stats(gdf, "data/filter1_stream.fgb", "data/AR50_flater.fgb")

        # Centroid for map center
        cent_proj = gdf.to_crs("EPSG:3857").geometry.union_all().centroid
        cent_wgs  = gpd.GeoSeries([cent_proj], crs="EPSG:3857") \
                        .to_crs("EPSG:4326").iloc[0]
        center = [cent_wgs.y, cent_wgs.x]

        gdf = gdf.to_crs("EPSG:4326")
        m = folium.Map(location=center, zoom_start=12)
        popup = folium.GeoJsonPopup(fields=["arealtype","områdeId"],
                                   aliases=["Arealtype:","Område-ID:"])
        folium.GeoJson(gdf, style_function=lambda f: {
            'fillColor':'blue','color':'blue','weight':1,'fillOpacity':0.5
        }, popup=popup).add_to(m)
        display(m)

# Wire up and display
load_button.on_click(on_click)
display(widgets.VBox([
    widgets.HBox([arealtype_widget, områdeid_widget, batch_slider]),
    load_button,
    output
]))

## Filtering: Tree species identification by coordinates: A radius search

### Cleaning the data.

In [ ]:
# Opprett tilkobling
con = get_duckdb_connection()

con.execute("""
CREATE OR REPLACE TEMPORARY VIEW renset_data AS
SELECT
  CASE WHEN arealtype IN (98, 99) THEN NULL ELSE arealtype END AS arealtype,
  CASE WHEN skogbonitet IN (98, 99) THEN NULL ELSE skogbonitet END AS skogbonitet,
  CASE WHEN treslag IN (98, 99) THEN NULL ELSE treslag END AS treslag,
  CASE WHEN jordbruk IN (98, 99) THEN NULL ELSE jordbruk END AS jordbruk,
  CASE WHEN vegetasjonsdekke IN (98, 99) THEN NULL ELSE vegetasjonsdekke END AS vegetasjonsdekke,
  *
FROM ST_READ('data/AR50_flater.fgb')
""")

### Query and visualization

In [37]:
# — Widgets for user input, including coordinates —
treslag_widget = widgets.IntText(
    value=0,
    description="Treslag (0=alle):",
    style={'description_width':'initial'}
)
buffer_widget = widgets.IntSlider(
    value=500, min=100, max=2000, step=100,
    description="Buffer (m):"
)
batch_widget = widgets.IntSlider(
    value=2, min=1, max=10, step=1,
    description="Batch-størrelse:"
)
lat_widget = widgets.FloatText(
    value=58.1743128719827,
    description="Breddegrad:",
    style={'description_width':'initial'}
)
lon_widget = widgets.FloatText(
    value=8.01122450699552,
    description="Lengdegrad:",
    style={'description_width':'initial'}
)
load_button = widgets.Button(description="Start streaming")
output = widgets.Output()

def on_click(_):
    with output:
        output.clear_output()
        treslag_val = treslag_widget.value
        buffer_m    = buffer_widget.value
        batch_size  = batch_widget.value
        lat         = lat_widget.value
        lon         = lon_widget.value

        # 1) Create buffer WKT around user-specified point
        pt = Point(lon, lat)
        buf_wkt = (
            gpd.GeoSeries([pt], crs="EPSG:4326")
               .to_crs("EPSG:25832")
               .iloc[0]
               .buffer(buffer_m)
               .wkt
        )
        print(f"Bruker buffer = {buffer_m} m rundt ({lat}, {lon})")

        # 2) Dynamic treslag filter
        if treslag_val != 0:
            clause = f"AND treslag = {treslag_val}\n"
            print(f"Filtrerer på treslag = {treslag_val}")
        else:
            clause = ""
            print("Ingen treslag-filtrering (0 = alle)")

        # 3) Build & execute query
        query = f"""
        SELECT
          gml_id,
          lokalId,
          treslag,
          områdeId,
          ST_AsText(
            ST_Intersection(geom, ST_GeomFromText('{buf_wkt}', false))
          ) AS geometry_wkt
        FROM renset_data
        WHERE treslag IS NOT NULL
          AND ST_Intersects(geom, ST_GeomFromText('{buf_wkt}', false))
        {clause}
        LIMIT 50;
        """
        print("\nStarter streaming...\n")
        batches = []
        for batch in stream_geo_data(con, query, batch_size=batch_size):
            batch['geometry'] = batch['geometry_wkt'].apply(wkt.loads)
            batches.append(batch)

        # 4) Combine & display
        result = pd.concat(batches, ignore_index=True)
        print(f"\nTotalt hentet {len(result)} rader.")
        display(result)

        # 5) GeoDataFrame, export & map
        gdf = gpd.GeoDataFrame(result, geometry='geometry', crs="EPSG:25832")
        export_gdf_with_stats(gdf, "data/filter2_stream.fgb", "data/AR50_flater.fgb")
        gdf = gdf.to_crs("EPSG:4326")

        # Center map on centroid
        centroid = (
            gdf.to_crs("EPSG:3857").geometry.union_all().centroid
        )
        centroid = gpd.GeoSeries([centroid], crs="EPSG:3857")\
                      .to_crs("EPSG:4326").iloc[0]

        m = folium.Map(location=[centroid.y, centroid.x], zoom_start=15)
        popup = folium.GeoJsonPopup(fields=['treslag'], aliases=['Treslag:'])
        folium.GeoJson(
            gdf,
            style_function=style_function,
            popup=popup
        ).add_to(m)
        display(m)

# Wire up and display
load_button.on_click(on_click)
display(widgets.VBox([
    treslag_widget,
    buffer_widget,
    batch_widget,
    lat_widget,
    lon_widget,
    load_button,
    output
]))